<a href="https://colab.research.google.com/github/AdrianOu-stats/Project2/blob/main/Project_2_Adrian_Ou_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_covid = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv")

In [ ]:
# Take a glance at the big picture of the data.

df_covid

In [ ]:
df_covid.info()

In [ ]:
df_covid.describe()

In [ ]:
# Found some null values. Try to loacte and clean them. Inspect "deaths" column first.

pd.isnull(df_covid['deaths'])

In [ ]:
pd.isnull(df_covid['deaths']).sum()

In [ ]:
df_covid[pd.isnull(df_covid['deaths'])]

In [ ]:
# Establish a deep copy to further investigate these null values in the section.

df_covid_copy_deaths = df_covid[pd.isnull(df_covid['deaths'])]

In [ ]:
df_covid_copy_deaths.info()

In [ ]:
df_covid_copy_deaths['state'].value_counts()

In [ ]:
# It seems like all null values in the "deaths" column are from Puerto Rico. 
# Puerto Rico's covid death counts hasn't been recorded for almost 16 months in this data frame. 37169 values are missing in total. 
# Due to the large numbers of missing values, I decided to exclude Puerto Rico's deaths counts when computing deaths counts related statistics.

In [ ]:
df_covid.cases.describe()

In [ ]:
df_covid

In [ ]:
# Next, some "fips" values are null. Let's investigate.

pd.isnull(df_covid['fips'])

In [ ]:
pd.isnull(df_covid['fips']).sum()

In [ ]:
df_covid[pd.isnull(df_covid['fips'])]

In [ ]:
# Accidentally found some "Unknown" values in "county" column. 
# However, one of my goals is to analyze New York state, so county info doesn't effect me for now.
# This reminded me to check if there are any unexpected values in "state" column. Checking now.

df_covid['state'].value_counts()

In [ ]:
# No unexpected value found. Proceed to investigate the null "fips" values.
# Establish another deep copy of this data frame with null 'fips value.

df_covid_copy_nullfips = df_covid[pd.isnull(df_covid['fips'])]

In [ ]:
# Take a cloer look at these null values.

df_covid_copy_nullfips.info()

In [ ]:
df_covid_copy_nullfips['state'].value_counts()

In [ ]:
# My focus is on New York, so I'm ignoring other states' null fips values.
# Attempt to replace NY null fips values with the correct ones.

## Let's start by getting a feel of this sectioned dataframe
df_covid_copy_nullfips[df_covid_copy_nullfips['state'] == 'New York'].sample(10)

In [ ]:
df_covid_copy_nullfips[df_covid_copy_nullfips['state'] == 'New York'].sample(10)

In [ ]:
df_covid_copy_nullfips[df_covid_copy_nullfips['state'] == 'New York'].head(40)

In [ ]:
df_covid_copy_nullfips[df_covid_copy_nullfips['state'] == 'New York'].tail(40)

In [ ]:
df_covid_copy_nullfips[df_covid_copy_nullfips['state'] == 'New York'].county.value_counts()

In [ ]:
df_covid_NYS = df_covid[df_covid['state'] == 'New York'].copy()
df_covid_NYS.info()

In [ ]:
df_covid_NYS[df_covid_NYS.county == 'Unknown']

In [ ]:
df_covid_NYS[df_covid_NYS.county == 'Unknown'].count()

In [ ]:
# 10 unknown counties with 10 NaN fips values, so no way to recover. 
# I want to further confirm that each of the county names has a one to one relationship with its fips value. If not, I will have to do some more cleaning.

df_covid_NYS[['county', 'fips']].value_counts()

In [ ]:
df_covid_NYS.county.value_counts().sum()

In [ ]:
df_covid_NYS.fips.value_counts().sum()

In [ ]:
# Of the 552 null values from New York State, 542 are from New York City and 10 are from unknown counties. 
# It should be safe to exclude the 10 unknown county values (out of 552) when computing county related statistics.
# Attempted to replace 542 null values with New York City fips number, but realized that the data doesn't specify "fips" values for any counties in New York City perhaps because it was contructed in a way that it views New York City as a whole.
# I decided to assign "36000.0" as New York City's "fips" code since the code "36000.0" hasn't been assigned to any counties yet.

df_covid_NYC = df_covid[df_covid['county'] == 'New York City'].copy()
df_covid_NYC

In [ ]:
df_covid_NYC.county.value_counts()

In [ ]:
# Null values are replaced by code 36000.0

df_covid_NYS.loc[df_covid_NYS.county == 'New York City', 'fips'] = 36000.0
df_covid_NYC = df_covid_NYS[df_covid_NYS['county'] == 'New York City'].copy()
df_covid_NYC.info()

In [ ]:
df_covid_NYS.info()

In [ ]:
df_covid_NYC['date'] = pd.to_datetime(df_covid_NYC.date)
df_covid_NYC_daily = df_covid_NYC.groupby('date',as_index=False, sort=False)[['cases', 'deaths']].sum()
df_covid_NYC_daily31 = df_covid_NYC_daily.iloc[-31:].copy()
df_covid_NYC_daily31['year'] = pd.DatetimeIndex(df_covid_NYC_daily31.date).year
df_covid_NYC_daily31['month'] = pd.DatetimeIndex(df_covid_NYC_daily31.date).month
df_covid_NYC_daily31['day'] = pd.DatetimeIndex(df_covid_NYC_daily31.date).day
df_covid_NYC_daily31

In [ ]:
df_covid_NYC_daily31['daily_new_cases'] = df_covid_NYC_daily31.cases.diff()
df_covid_NYC_daily31['daily_new_deaths'] = df_covid_NYC_daily31.deaths.diff()
df_covid_NYC_daily30 = df_covid_NYC_daily31.iloc[-30:].copy()
df_covid_NYC_daily30

In [ ]:
df_covid_NYC_daily30.describe()

In [ ]:
plt.plot(df_covid_NYC_daily30.date, df_covid_NYC_daily30.daily_new_cases)
plt.gcf().set_size_inches(15, 10);

In [ ]:
plt.plot(df_covid_NYC_daily30.date, df_covid_NYC_daily30.daily_new_deaths)
plt.gcf().set_size_inches(15, 10);

**Next, I want to look at the NYS in the past 30 days.**

**I want to monitor the recent changes for each county and then see NYS as a whole.**

**Two major goals here**
1.   Use slicing and filtering to get the past 31 days dataframe. 
2.   Derive new daily cases and new daily deaths from cumulative cases and deaths. Adjust dataframe from 31 to 30 days.





In [ ]:
# Convert the datatype from string to datetime 
df_covid_NYS['date'] = pd.to_datetime(df_covid_NYS.date)
df_covid_NYS.info()

In [ ]:
# I want to add a year, a month, and a day column.
df_covid_NYS['year'] = pd.DatetimeIndex(df_covid_NYS.date).year
df_covid_NYS['month'] = pd.DatetimeIndex(df_covid_NYS.date).month
df_covid_NYS['day'] = pd.DatetimeIndex(df_covid_NYS.date).day
df_covid_NYS

In [ ]:
# Create a "df_covid_NYS_copy" excluding the 10 unknowns so that I can better analyze each indiviual county in NYS now.
# This way "df_covid_NYS" is preserved for state wise analysis later on.
df_covid_NYS_copy = df_covid_NYS[df_covid_NYS.county != 'Unknown'].copy()
df_covid_NYS_copy

In [ ]:
# Filter to get the past 31 days dataframe.
df_covid_NYS_21 = df_covid_NYS_copy[df_covid_NYS_copy.year == 2021].copy() 
df_covid_NYS_21_0723 = df_covid_NYS_21[(100*df_covid_NYS_21.month + df_covid_NYS_21.day) >= (700+23)].copy()
df_covid_NYS_21_0723_to_0822 = df_covid_NYS_21_0723[(100*df_covid_NYS_21_0723.month + df_covid_NYS_21_0723.day) <= (800+22)].copy()

In [ ]:
# Sort. Preparing for the diff() moment.
df_covid_NYS_21_0723_to_0822_sort = df_covid_NYS_21_0723_to_0822.sort_values(['county', 'date']).copy()
df_covid_NYS_21_0723_to_0822_sort

In [ ]:
# Perform diff() to derive new daily counts. 
df_covid_NYS_21_0723_to_0822_sort['daily_new_cases'] = df_covid_NYS_21_0723_to_0822_sort.cases.diff()
df_covid_NYS_21_0723_to_0822_sort['daily_new_deaths'] = df_covid_NYS_21_0723_to_0822_sort.deaths.diff()

In [ ]:
# Double checking these sacrificial rows.
rows_20210723 = df_covid_NYS_21_0723_to_0822_sort[df_covid_NYS_21_0723_to_0822_sort.date == '2021-07-23']
rows_20210723

In [ ]:
# 62 counties in NYS where 5 counties are counted as one here. 
rows_20210723.count()

In [ ]:
# Adjusting dataframe from 31 days to 30 days by excluding those invalid rows resulted from the diff() function.
df_covid_NYS_21_0724_to_0822_sort = df_covid_NYS_21_0723_to_0822_sort[
                                                                      (100*df_covid_NYS_21_0723_to_0822_sort.month 
                                                                       + df_covid_NYS_21_0723_to_0822_sort.day)
                                                                       >(700+23)].copy()
df_covid_NYS_21_0724_to_0822_sort

In [ ]:
df_covid_NYS_21_0724_to_0822_sort.info()

**Unfortunately, I have to admit that it was overly ambitious to try to analyze the trend of 58 counties with the skillset I currently possess. This is as far as I can go for now. I will revisit this attempt in the near future.**

In [ ]:
# Let's take a look at NYS as a whole. I want to see the past 30 days daily data for NYS. 

df_covid_NYS_daily = df_covid_NYS.groupby('date',as_index=False, sort=False)[['cases', 'deaths']].sum()
df_covid_NYS_daily31 = df_covid_NYS_daily.iloc[-31:].copy()
df_covid_NYS_daily31.describe()

In [ ]:
plt.plot(df_covid_NYS_daily31.date, df_covid_NYS_daily31.cases)
plt.gcf().set_size_inches(15, 15)

In [ ]:
plt.plot(df_covid_NYS_daily31.date, df_covid_NYS_daily31.deaths)
plt.gcf().set_size_inches(15, 15)

In [ ]:
# The "cases" and "deaths" columns are cumulative counts. They either go up or stay flat. I can't tell too much from them.
# They are not too clear in relecting the recent changes, so I want to find the new cases and deaths per day.

df_covid_NYS_daily31['daily_new_cases'] = df_covid_NYS_daily31.cases.diff()
df_covid_NYS_daily31['daily_new_deaths'] = df_covid_NYS_daily31.deaths.diff()
df_covid_NYS_daily31['month'] = pd.DatetimeIndex(df_covid_NYS_daily31.date).month
df_covid_NYS_daily31['day'] = pd.DatetimeIndex(df_covid_NYS_daily31.date).day
df_covid_NYS_daily30 = df_covid_NYS_daily31.iloc[-30:].copy()
df_covid_NYS_daily30

In [ ]:
df_covid_NYS_daily30.describe()

In [ ]:
# So much better. Let's plot.

plt.plot(df_covid_NYS_daily30.date, df_covid_NYS_daily30.daily_new_cases)
plt.gcf().set_size_inches(10, 10)

In [ ]:
plt.plot(df_covid_NYS_daily30.date, df_covid_NYS_daily30.daily_new_deaths)
plt.gcf().set_size_inches(10, 10)

In [ ]:
# Comparing the two other states to NYS. 
# I selected two states with various controversial COVID and COVID vaccine related governmental policies/attitudes. 
# Florida and Califlornia

df_covid_FL = df_covid[df_covid['state'] == 'Florida'].copy()
df_covid_FL['date'] = pd.to_datetime(df_covid_FL.date)
df_covid_CA = df_covid[df_covid['state'] == 'California'].copy()
df_covid_CA['date'] = pd.to_datetime(df_covid_CA.date)

In [ ]:
df_covid_FL

In [ ]:
df_covid_FL.info()

In [ ]:
df_covid_CA

In [ ]:
df_covid_CA.info()

In [ ]:
# Group and slice.
df_covid_FL_daily = df_covid_FL.groupby('date',as_index=False, sort=False)[['cases', 'deaths']].sum()
df_covid_FL_daily31 = df_covid_FL_daily.iloc[-31:].copy()
df_covid_CA_daily = df_covid_CA.groupby('date',as_index=False, sort=False)[['cases', 'deaths']].sum()
df_covid_CA_daily31 = df_covid_CA_daily.iloc[-31:].copy()

In [ ]:
# Derive and exclude. (FL)
df_covid_FL_daily31['daily_new_cases'] = df_covid_FL_daily31.cases.diff()
df_covid_FL_daily31['daily_new_deaths'] = df_covid_FL_daily31.deaths.diff()
df_covid_FL_daily30 = df_covid_FL_daily31.iloc[-30:].copy()
df_covid_FL_daily30['month'] = pd.DatetimeIndex(df_covid_FL_daily30.date).month
df_covid_FL_daily30['day'] = pd.DatetimeIndex(df_covid_FL_daily30.date).day
df_covid_FL_daily30

In [ ]:
df_covid_FL_daily30.describe()

In [ ]:
# Derive and exclude. (CA)
df_covid_CA_daily31['daily_new_cases'] = df_covid_CA_daily31.cases.diff()
df_covid_CA_daily31['daily_new_deaths'] = df_covid_CA_daily31.deaths.diff()
df_covid_CA_daily31['month'] = pd.DatetimeIndex(df_covid_CA_daily31.date).month
df_covid_CA_daily31['day'] = pd.DatetimeIndex(df_covid_CA_daily31.date).day
df_covid_CA_daily30 = df_covid_CA_daily31.iloc[-30:].copy()
df_covid_CA_daily30

In [ ]:
# There are two negative values (-5 and -357) in the daily_new_deaths column. Presumably, they are used to offset past recording errors. 
# This time I am replacing both with the mean value that is the average of the past 30 days excluding the data point -357.
# It makes sense not to adjust the cumulative death counts since changing it defeats the very purpose of the correction.

# Compute mean:
CA_new_deaths_mean = df_covid_CA_daily30.daily_new_deaths[df_covid_CA_daily30.daily_new_deaths >= -5].mean()
CA_new_deaths_mean

In [ ]:
# Replace:
df_covid_CA_daily30.loc[556, 'daily_new_deaths'] = CA_new_deaths_mean
df_covid_CA_daily30.loc[564, 'daily_new_deaths'] = CA_new_deaths_mean
df_covid_CA_daily30

In [ ]:
df_covid_CA_daily30.describe()

In [ ]:
# Refresh memory.
df_covid_NYS_daily30

In [ ]:
df_covid_NYS_daily30.describe()

**Let's look at the cumulative cases and deaths counts per day for FL, CA, and NYS.**

In [ ]:
plt.plot(df_covid_FL_daily30.date, df_covid_FL_daily30.cases, label="FL")
plt.plot(df_covid_CA_daily30.date, df_covid_CA_daily30.cases, label="CA")
plt.plot(df_covid_NYS_daily30.date, df_covid_NYS_daily30.cases, label="NYS")
plt.legend()
plt.gcf().set_size_inches(15, 10)
plt.title("Cumulative Cases Reported Past 30 days");

In [ ]:
plt.plot(df_covid_FL_daily30.date, df_covid_FL_daily30.deaths, label="FL")
plt.plot(df_covid_CA_daily30.date, df_covid_CA_daily30.deaths, label="CA")
plt.plot(df_covid_NYS_daily30.date, df_covid_NYS_daily30.deaths, label="NYS")
plt.legend()
plt.gcf().set_size_inches(15, 10)
plt.title("Cumulative Deaths Reported Past 30 days");

**Now the new cases and deaths counts per day for FL, CA, and NYS.**

In [ ]:
plt.plot(df_covid_FL_daily30.date, df_covid_FL_daily30.daily_new_cases, label="FL")
plt.plot(df_covid_CA_daily30.date, df_covid_CA_daily30.daily_new_cases, label="CA")
plt.plot(df_covid_NYS_daily30.date, df_covid_NYS_daily30.daily_new_cases, label="NYS")
plt.legend()
plt.gcf().set_size_inches(15, 10)
plt.title("New Daily Cases Reported Past 30 days");

In [ ]:
plt.plot(df_covid_FL_daily30.date, df_covid_FL_daily30.daily_new_deaths, label="FL")
plt.plot(df_covid_CA_daily30.date, df_covid_CA_daily30.daily_new_deaths, label="CA")
plt.plot(df_covid_NYS_daily30.date, df_covid_NYS_daily30.daily_new_deaths, label="NYS")
plt.legend()
plt.gcf().set_size_inches(15, 10)
plt.title("New Daily Deaths Reported Past 30 days");

Conclusion:

1.  Perform preliminary checks on the original dataframe. Found missing values in columns "deaths" and "fips".  After investigating, proceed to ignore missing "deaths" values due to their irrelevance to our project. Attempt to fix abnormal values in column "fips" to hopefully gain more insight on the intrastate situation, New York state specifically. 

2. Get a better understanding on both said columns by using functions such as, .info(), .value_counts(), etc. Recognize all NaN fips values are linked to NYC. Replace NaN values with 36000.0 indicating values are from NYC as a whole. Found non-typical null values (i.e., Unknown) in column "county". Make a copy of the dataframe excluding the 10 unknown county values for intrastate analysis. Clean and take a look at NYC data in the past 30 days.

3. Take a look at the intrastate situation: slice and filter the dataframe copy to get the restricted dataframe that only contains only New York state data in the past 31 days. Rearrange the data and derive the new cases and deaths per day for each county. Decide to abandon this intrastate analysis due to lacking or forgetting the mass graphing/analysis techniques to effectively analyze the data of 58 counties. Will revisit in the future.

4. Take a look at NYS as a whole: group dataframe by "date" and slice it to get the past 30 days in sequential order. However, the cumulative counts don't provide too much insight into the recent situation, so I further derive the daily new cases and deaths by using diff() function. After cleaning the past 30 days' data, I find the following. Daily new cases: mean = 4173, mode = 3972, and standard deviation = 1323; daily new deaths: mean = 19, mode = 16, and standard deviation = 9.8. Combined with graphs, I have the overall picture of the trend in NYS. There seems to be a trend of cases and deaths rising; however, the slope of both graphs looks relatively flat. Despite having about 4k new cases daily (suggested by both mean and mode), the death counts per day are rather low, probably due to having the highest vaccination rate in the country.

5. Compare two states (i.e., California and Florida) to NYS. Apply similar treatment to extract the dateframes of both states in the past 30 days.

6. The way I interpret the case and death counts is that the case counts likely reveal how well the state prevents COVID19 from spreading and that the death counts show how strong the patients in fighting against the disease. All three states have daily case counts increased; however, FL and CA have significantly more average new cases and deaths than NYS. FL has a whopping 182.7 new daily death counts in the past 30 days and the highest 901 death count for a single day. CA is better but not as good as NYS's low new case counts and much lower deaths per day. I think the protective restrictions and the vaccination rates play an important role in flatting the spread curve as well as in lowering the death rates.

